# Clustering

In [38]:
import pandas as pd
import numpy as np
import warnings
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

### Przygotowanie danych i wybór zmiennych

In [39]:
df = pd.read_csv('data/bmd_prep.csv')

numeric_predictors = df.select_dtypes(include=np.number).columns.tolist()
categorical_predictors =  df.select_dtypes(exclude=np.number).columns.tolist()
# exclude less important predictors
categorical_predictors.remove('sex')
numeric_predictors.remove('height_cm')
numeric_predictors.remove('waiting_time')

X = df[numeric_predictors + categorical_predictors]

# one-hot encode categorical variables
X = pd.get_dummies(X)

# Standarize data
scaler = MinMaxScaler()
X[numeric_predictors] = scaler.fit_transform(X[numeric_predictors])
X.head()

,age,weight_kg,bmd,fracture_fracture,fracture_no fracture,medication_Anticonvulsant,medication_Glucocorticoids,medication_No medication
0,0.401187,0.466667,0.494030,0,1,1,0,0
1,0.754200,0.700000,0.405320,0,1,0,0,1
2,0.660465,0.616667,0.522727,0,1,0,0,1
3,0.801536,0.400000,0.317972,0,1,0,0,1
4,0.347146,0.316667,0.401445,0,1,0,0,1


Do analizy zostały wybrane wszystkie zmienne oprócz `sex`, `height_cm` oraz `waiting_time`.
* Zmienną `sex` można odrzucić, gdyż jej ważność określona przy okazji testu Chi kwadrat lub przy indukcji drzew decyzyjnych okazała się bardzo mała. Nie obsewowano dużej różnicy między mężczyznami a kobietami.
* Zmienną `height_cm` oraz `waiting_time` można odrzucić, ze względu na potrzebę redukcji wymiarowości. Innym powodem jest fakt, iż te zmienne nie zostały wzięte pod uwagę przy definiowaniu hipotez. Zmienna `waiting_time` nie jest dokładnie określona, co oznacza.

### Optimize number of clusters

In [40]:
warnings.filterwarnings('ignore')

def custom_scorer(estimator,X):
    return silhouette_score(X, estimator.predict(X), metric='euclidean')

# KMeans object
km = KMeans(init='k-means++', n_init=10, max_iter=50)

# List of possible number of clusters
param = range(2,10)

# Sets up GridSearchCV object and stores in grid variable
grid = GridSearchCV(
    km,
    {'n_clusters': param},
    scoring=custom_scorer,
    cv=10) # cross validations

# Fits the grid object to data
grid.fit(X)

# Accesses the optimum model
best_km = grid.best_estimator_

# Displays the optimum model
best_km.get_params()

{'algorithm': 'lloyd',
 'copy_x': True,
 'init': 'k-means++',
 'max_iter': 50,
 'n_clusters': 5,
 'n_init': 10,
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

## 2. Clustering - K-means method

In [41]:
params = best_km.get_params()

# kMeans model
# km = KMeans(
#     n_clusters=4,
#     init='k-means++',
#     n_init=10,
#     max_iter=50)
km = KMeans(**params)

# Fits the model to the data
km.fit(X)

km.get_params()

{'algorithm': 'lloyd',
 'copy_x': True,
 'init': 'k-means++',
 'max_iter': 50,
 'n_clusters': 5,
 'n_init': 10,
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

### Basic statistics of obtained clusters - numeric features

In [42]:
km_results = df[numeric_predictors]
km_results['Cluster'] = km.labels_

km_results.groupby('Cluster')[numeric_predictors].agg(['min', 'max', 'mean', 'std'])

age                                  weight_kg        \
               min        max       mean        std       min   max   
Cluster                                                               
0        50.276059  86.859869  73.803475  13.105046      40.0  79.0   
1        35.814058  84.787394  60.776615  11.216124      39.0  96.0   
2        37.461995  88.024151  68.886213  13.436667      36.0  85.0   
3        43.450712  88.753795  62.691945  10.998231      47.0  90.0   
4        56.931266  70.421873  60.047460   5.823149      51.0  74.0   

                                  bmd                              
              mean        std     min     max      mean       std  
Cluster                                                            
0        57.333333  10.665365  0.4076  0.7399  0.566000  0.107245  
1        67.505263  10.608982  0.5514  1.2508  0.857752  0.129653  
2        58.646341  11.174885  0.4407  0.8664  0.635888  0.097970  
3        67.947368  11.896422  0.5090  1.3624  0.836605  0.176797  
4        60.800000   8.871302  0.5899  1.0287  0.759460  0.187695

### Basic statistics of obtained clusters - categorical features

In [43]:
km_results = df[categorical_predictors]
km_results['Cluster'] = km.labels_

for predictor in categorical_predictors:
    counter_table = km_results.groupby(['Cluster', predictor])[predictor].agg(['count'])
    s = counter_table.style.set_caption(f'Counter table for categorical predictor: {predictor}')
    display(s)

,,count
Cluster,fracture,
0,fracture,9
1,no fracture,95
2,fracture,41
3,no fracture,19
4,no fracture,5


## 3. Interpretacja wyników - metoda K-means

Otrzymane skupienia są jednorodne pod względem wystąpienia złamania (`fracture`)

##### Skupienie 1 (0):
* najstarsza grupa wiekowa spośród uzyskanych, średnia wieku 73 lata
* osoby, które średnio ważą najmniej
* najmniejsza średnia wartość wskaźnika `bmd` (z kośćmi najmniej bogatymi w minerały)
* są to osoby, które pomimo leczenia, doznały złamania
##### Skupienie 2:
* najliczniejsza i jedna z najmłodszych otrzymanych grup
* osoby cechują się stosunkowo dużą wagą, choć zdarzają się osoby o małej wadze
* mają zdrowe kości - `bmd` na wysokim poziomie
* są to osoby zdrowe, gdyż nie doznały złamania i nie były leczone
##### Skupienie 3:
* skupienie podobne do skupienia 1 pod względem wiekowym, wagowym i wskaźnika `bmd`
* W przeciwieństwie do skupienia 1, gdzie osoby były leczone, w tym skupieniu osoby nie były leczone
* osoby te doznały złamania - najprawdopodobniej spowodowane słabymi kośćmi (niskie `bmd`)
##### Skupienie 4:
* grupa osób młodszych w stosunku do pozostałych, o średnio dużej wadzę
* duża ilość minerałów w kościach
* nie doznali złamania, leczeni przy pomocy glukokortykozu (można tą grupę interpretować jako tych co dzięki lekom nie doznali złamania)
##### Skupienie 5:
* ludzie, którzy nie doznali złamania
* stosunkowo mała waga
* leczeni przy pomocy leków przeciwdrgawkowych, nie doznali złamań

##### Odniesienie do hipotez
Otrzymane skupienia potwierdzają zgodność hipotez. Widać, że wraz ze wzrostem wieku wartość `bmd` spada, a im większa waga tym większe `bmd`. W skupieniach 2 i 4 znajdują się osoby, które ważą dużo, są stosunkowo młodsze (ok. 60 lat) i rzeczywiście ich wskaźnik `bmd` jest na wysokim poziomie. Jako kontrast, w skupieniach 1 i 3 znajdują się osoby, które są starsze (ok. 70 lat), ważą mniej niż w skupieniach 2 i 4, a ich wskaźnik `bmd` jest niższy. Hipoteza 2 jest również słuszna - ludzie z niskim `bmd` i starsi doznają na ogół złamań (jak w skupieniach 1 i 3), natomiast ludzie, u których `bmd` ma wysoką wartość i są młodsze (skupienia 2 i 4) - na ogól nie doznają złamań. Ciekawą, wyróżniającą się grupą jest skupienie 5, u których waga i wskaźnik `bmd` nie są wcale największe a mimo to nie doznali złamań. Można wnioskować, że podane pacjentom leki (anticonvulsant) sprawiły, że uniknęły one złamania. Zatem leki mogą mieć wpływ na wystąpienie złamania (jak założono w hipotezie 2).

## 4. Clustering - EM method

In [55]:
gmm = GaussianMixture(n_components=3)

# Fit model to data
gmm.fit(X)

# Predict cluster labels for data
labels = gmm.predict(X)

### EM method - results (numeric features)

In [58]:
em_results = df[numeric_predictors]
em_results['Cluster'] = labels

em_results.groupby('Cluster')[numeric_predictors].agg(['min', 'max', 'mean', 'std'])

age                                  weight_kg        \
               min        max       mean        std       min   max   
Cluster                                                               
0        35.814058  84.787394  60.776615  11.216124      39.0  96.0   
1        43.450712  88.753795  62.141010  10.087913      47.0  90.0   
2        37.461995  88.024151  69.771320  13.381485      36.0  85.0   

                                  bmd                              
              mean        std     min     max      mean       std  
Cluster                                                            
0        67.505263  10.608982  0.5514  1.2508  0.857752  0.129653  
1        66.458333  11.542847  0.5090  1.3624  0.820533  0.177801  
2        58.410000  10.989647  0.4076  0.8664  0.623308  0.102219

### EM method - results (categorical features)

In [57]:
em_results = df[categorical_predictors]
em_results['Cluster'] = labels

for predictor in categorical_predictors:
    counter_table = em_results.groupby(['Cluster', predictor])[predictor].agg(['count'])
    s = counter_table.style.set_caption(f'Counter table for categorical predictor: {predictor}')
    display(s)

,,count
Cluster,fracture,
0,no fracture,95
1,no fracture,24
2,fracture,50


## 5. EM method - charakterystyka skupień

Otrzymane skupienia, podobnie jak w metodzie k-średnich są jednorodne pod względem wystąpienia złamania bądź nie. Jednak różnią się między sobą m.in. pod względem sposobu leczenia.

##### Skupienie 1
* osoby, które nie były leczone, nie doznały złamań, a ich kości mają wysoki poziom minerałów (duża wartość `bmd`)
* osoby, które ważą średnio najwięcej spośród otrzymanych skupień oraz są średnio najmłodsi
##### Skupienie 2
* skupienie dośc podobne do skupienia 1 pod względem wieku, wagi i `bmd`
* mniej liczne niż skupienie 1, osoby leczone, które nie doznały złamań
##### Skupienie 3
* skupienie osób zarówno leczonych jak i nie (które stanowią większość).
* najstarsza grupa osób, z najsłabszymi koścmi, którzy doznali złamania
